## LangChain Agents - A Comprehensive Guide

### What are Agents?

**Agents** are one of the most powerful concepts in LangChain. Unlike simple chains that follow a predetermined sequence, agents use an LLM as a **reasoning engine** to dynamically determine:
1. **Which actions** to perform
2. **In what order** to perform them
3. **When to stop** and return a final answer

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Agent** | The decision-maker that uses an LLM to reason about what to do next |
| **Tools** | Functions that agents can call to interact with the external world |
| **Agent Executor** | The runtime that orchestrates the agent's reasoning and tool execution loop |
| **Scratchpad** | Memory of previous actions and observations within a single run |

### The ReAct Pattern

Most agents follow the **ReAct (Reasoning + Acting)** pattern:
1. **Thought**: The agent reasons about what to do
2. **Action**: The agent decides which tool to use
3. **Action Input**: The input to pass to the tool
4. **Observation**: The result from the tool
5. **Repeat** until the agent has enough information to provide a final answer

> ⚠️ **Note**: LangChain is transitioning to **LangGraph** for building agents. However, understanding these concepts is still valuable as they form the foundation.

---

## Section 1: Understanding Tools

### What are Tools?

**Tools** are functions that agents can use to interact with the external world. They extend the agent's capabilities beyond just generating text.

### Types of Tools

| Type | Examples | Use Case |
|------|----------|----------|
| **Built-in Tools** | `llm-math`, `wikipedia`, `serpapi` | Common utilities provided by LangChain |
| **Custom Tools** | Your own functions | Domain-specific operations |
| **Chain Tools** | Other LangChain chains | Complex multi-step operations |
| **Agent Tools** | Other agents | Hierarchical agent systems |

### Tool Components

Every tool has:
- **name**: A unique identifier for the tool
- **description**: Explains what the tool does (crucial for the agent to decide when to use it!)
- **func**: The actual function that gets executed

> 💡 **Pro Tip**: The `description` is extremely important! The agent uses it to decide which tool to use. Make it clear and specific.

In [24]:
# =============================================================================
# STEP 1: Environment Setup
# =============================================================================
# Load environment variables from .env file
# This typically includes API keys like OPENAI_API_KEY
# The find_dotenv() function searches for .env file in parent directories

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  # Returns True if .env file was found and loaded

True

In [25]:
# =============================================================================
# STEP 2: Loading Built-in Tools
# =============================================================================
# LangChain provides several pre-built tools that you can use out of the box
# Here we're loading the "llm-math" tool - a calculator that uses an LLM

from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

# Initialize the LLM that will power our agent
# gpt-4o-mini is a cost-effective model suitable for agent tasks
llm = ChatOpenAI(model="gpt-4o-mini")

# Load built-in tools by name
# "llm-math" - Uses an LLM to solve math problems by generating Python expressions
# Other available tools: "wikipedia", "serpapi", "requests", "terminal", etc.
tool_names = ["llm-math"]

# Some tools require an LLM to function (like llm-math which translates
# natural language math problems into executable expressions)
tools = load_tools(tool_names, llm=llm)

# Let's inspect what we got - a list of Tool objects
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [26]:
# =============================================================================
# STEP 3: Creating an Agent with initialize_agent
# =============================================================================
# The initialize_agent function creates an agent that can use the provided tools
# This is the "legacy" way - LangGraph is the recommended approach for new projects

from langchain.agents import initialize_agent

# Create the agent with the following parameters:
agent = initialize_agent(
    tools,                                      # List of tools the agent can use
    llm,                                        # The LLM to use for reasoning
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type (see below)
    verbose=True,                               # Print agent's thought process
    max_iterations=3,                           # Prevent infinite loops
    handle_parsing_errors=True                  # Gracefully handle LLM output parsing errors
)

# =============================================================================
# Agent Types Explained:
# =============================================================================
# ZERO_SHOT_REACT_DESCRIPTION:
#   - Uses ReAct pattern (Reasoning + Acting)
#   - "Zero-shot" means no examples are provided
#   - Decides which tool to use based on tool descriptions
#
# Other agent types:
#   - CONVERSATIONAL_REACT_DESCRIPTION: For chat with memory
#   - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION: For complex tool inputs
#   - OPENAI_FUNCTIONS: Uses OpenAI's function calling (more reliable)
# =============================================================================

In [27]:
# =============================================================================
# Inspecting the Agent's Components
# =============================================================================
# The agent has a nested structure - let's explore it

# Access the underlying LLM model name
# agent.agent -> The actual agent logic
# agent.agent.llm_chain -> The chain that calls the LLM
# agent.agent.llm_chain.llm -> The LLM instance
print(agent.agent.llm_chain.llm.model_name)

gpt-4o-mini


In [28]:
# Print the prompt template that guides the agent's behavior
# This template is dynamically constructed based on the tools provided
print(agent.agent.llm_chain.prompt.template)

# =============================================================================
# OUTPUT BREAKDOWN:
# =============================================================================
# 1. SYSTEM INSTRUCTIONS: "Answer the following questions as best you can..."
# 2. TOOL DESCRIPTIONS: Lists available tools and their descriptions
# 3. FORMAT INSTRUCTIONS: The ReAct format (Thought/Action/Observation)
# 4. PLACEHOLDERS:
#    - {input}: Where the user's question goes
#    - {agent_scratchpad}: Previous thoughts/actions in current run
# =============================================================================

Answer the following questions as best you can. You have access to the following tools:

Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Understanding the ReAct Prompt Template

The ReAct (Reasoning + Acting) pattern is the foundation of how agents work. The prompt template instructs the LLM on:
1. **How to think** through the problem step by step
2. **When to use tools** based on their descriptions
3. **How to format responses** so they can be parsed correctly
4. **When to stop** and provide a final answer

Let's examine the actual prompt template used by our agent. Notice the structured format with `Thought`, `Action`, `Action Input`, and `Observation` sections:

### Example 1: Agent Using Its Knowledge (No Tool Needed)

Let's ask the agent a question that doesn't require the calculator tool. Watch how the agent reasons and decides it already knows the answer:


In [30]:
# =============================================================================
# Running the Agent - General Knowledge Question
# =============================================================================
# Notice in the output:
# - The agent recognizes this is a general knowledge question
# - It doesn't need to use the Calculator tool
# - It goes straight to "Final Answer" after thinking

result = agent.invoke(input="How many members does the A Team have?")

# The result is a dictionary containing:
# - 'input': The original question
# - 'output': The agent's final answer
result



> Entering new AgentExecutor chain...
The question asks about the number of members in the A Team. I need to recall or find out how many members are typically associated with the A Team. 

Action: None (I will answer based on knowledge)
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:The A Team is a fictional group of characters from the television show and movie of the same name. The original A Team consists of four main members:

1. Colonel John "Hannibal" Smith
2. Lieutenant Templeton "Faceman" Peck
3. Captain H.M. "Howling Mad" Murdock
4. Sergeant Bosco "B.A." Baracus

Thought: I now know the final answer
Final Answer: The A Team has four members.

> Finished chain.


{'input': 'How many members does the A Team have?',
 'output': 'The A Team has four members.'}

### Example 2: Agent Using a Tool (Calculator)

Now let's ask a math question. Watch how the agent:
1. **Thinks** about what to do
2. **Decides** to use the Calculator tool
3. **Provides input** to the tool
4. **Observes** the result
5. **Formulates** the final answer


In [31]:
# =============================================================================
# Running the Agent - Math Question (Requires Tool)
# =============================================================================
# The verbose output shows the complete reasoning chain:
#
# > Entering new AgentExecutor chain...
# Thought: I need to perform division
# Action: Calculator
# Action Input: 100 / 25
# Observation: Answer: 4.0
# Thought: I now know the final answer
# Final Answer: 4.0
# > Finished chain.

agent.invoke(input="What is 100 devided by 25?")

# Note: Even with the typo "devided", the agent understands the intent!



> Entering new AgentExecutor chain...
To find the result of 100 divided by 25, I will perform the division calculation.  
Action: Calculator  
Action Input: 100 / 25  
Observation: Answer: 4.0
Thought:I now know the final answer.  
Final Answer: 4.0

> Finished chain.


{'input': 'What is 100 devided by 25?', 'output': '4.0'}

---

## Section 2: Creating Custom Tools

### Why Custom Tools?

Built-in tools are great, but real-world applications often need domain-specific functionality. You can create custom tools to:
- Query your own databases
- Call internal APIs
- Perform specialized computations
- Access proprietary data sources

### Ways to Create Custom Tools

| Method | Complexity | Use Case |
|--------|------------|----------|
| `@tool` decorator | Simple | Quick functions with no state |
| `BaseTool` subclass | Medium | When you need more control |
| `StructuredTool` | Advanced | Complex inputs with validation |

### Example: Restaurant Search Tool

Let's create a custom tool that searches a FAISS vector store containing restaurant information:

In [32]:
# =============================================================================
# STEP 1: Load the Vector Store (Knowledge Base)
# =============================================================================
# First, we need a data source for our custom tool
# Here we're loading a pre-built FAISS vector store containing restaurant FAQ data

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize the same embeddings model used to create the index
embeddings = OpenAIEmbeddings()

# Load the pre-built FAISS index from disk
# The 'index' folder contains:
#   - index.faiss: The actual vector index
#   - index.pkl: Metadata and document mappings
# 
# ⚠️ allow_dangerous_deserialization=True is required because loading
# pickled data can be a security risk if from untrusted sources
vectorstore = FAISS.load_local("index", embeddings, allow_dangerous_deserialization=True)

In [34]:
# =============================================================================
# STEP 2: Define a Custom Tool Using BaseTool
# =============================================================================
# Creating a custom tool by subclassing BaseTool gives you full control
# over the tool's behavior and allows for more complex logic

from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun


class CustomSearchTool(BaseTool):
    """
    A custom tool that searches a restaurant FAQ vector store.
    
    When the agent needs information about the restaurant, it will use this tool
    to retrieve relevant information from the vector store.
    """
    
    # REQUIRED: Tool name - used by the agent to reference this tool
    name: str = "restaurant search"
    
    # REQUIRED: Description - CRITICAL! The agent uses this to decide when to use the tool
    # Make it clear and specific about what the tool does
    description: str = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """
        The main method that gets called when the agent uses this tool.
        
        Args:
            query: The search query from the agent
            run_manager: Optional callback manager for tracking
            
        Returns:
            String containing relevant information from the vector store
        """
        # Convert vectorstore to a retriever
        store = vectorstore.as_retriever()
        
        # Perform semantic search
        docs = store.invoke(query)
        
        # Extract text content from retrieved documents
        text_list = [doc.page_content for doc in docs]
        
        # Return combined results
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """
        Async version of the tool (required by BaseTool but not implemented here).
        Raise NotImplementedError if async is not supported.
        """
        raise NotImplementedError("custom_search does not support async")

In [44]:
# =============================================================================
# STEP 3: Create an Agent with Our Custom Tool
# =============================================================================
# Now we instantiate our custom tool and create a new agent
# This agent will ONLY have access to the restaurant search tool

from langchain.agents import AgentType

# Create an instance of our custom tool
# We can create multiple tools and pass them as a list
tools = [CustomSearchTool()]

# Initialize a new agent with our custom tool
# Using the same ZERO_SHOT_REACT_DESCRIPTION pattern
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,               # Show the agent's thinking process
    max_iterations=3,           # Limit iterations to prevent infinite loops
    handle_parsing_errors=True  # Handle malformed LLM outputs gracefully
)

### Testing Our Custom Tool Agent

#### Test 1: Question Outside the Tool's Scope

Let's first ask a question that the restaurant tool can't answer. The agent should recognize this and use its own knowledge:


In [45]:
# =============================================================================
# Test 1: Out-of-Scope Question
# =============================================================================
# The agent only has the restaurant search tool, but this question is about
# a TV show. The agent should:
# 1. Recognize the tool won't help
# 2. Use its own knowledge to answer
#
# Notice how the agent doesn't waste time searching for irrelevant info!

agent.invoke(input="Who are the members of the A-Team?")



> Entering new AgentExecutor chain...
The A-Team is a fictional group from the television series of the same name. I need to recall the main characters and their roles. The primary members include:

1. John "Hannibal" Smith - the leader and strategist.
2. Templeton "Faceman" Peck - the conman and smooth talker.
3. B.A. Baracus - the strongman and mechanic.
4. H.M. Murdock - the pilot and often considered the crazy character.

Thought: I now know the final answer
Final Answer: The members of the A-Team are John "Hannibal" Smith, Templeton "Faceman" Peck, B.A. Baracus, and H.M. Murdock.

> Finished chain.


{'input': 'Who are the members of the A-Team?',
 'output': 'The members of the A-Team are John "Hannibal" Smith, Templeton "Faceman" Peck, B.A. Baracus, and H.M. Murdock.'}

#### Test 2: Question Within the Tool's Scope

Now let's ask about the restaurant. The agent should use our custom tool.

---

### 🔧 Common Issue: "Agent stopped due to iteration limit"

If you see this error, here's what's happening and how to fix it:

**Root Cause**: The vector store returns fragmented results (questions without answers), causing the agent to keep searching.

**Fixes (in order of effectiveness):**

| Fix | Where | Code Change |
|-----|-------|-------------|
| 1. Retrieve more docs | `as_retriever()` | `search_kwargs={"k": 6}` |
| 2. Better formatting | `_run()` return | Add clear headers like "Information Found:" |
| 3. Increase iterations | `initialize_agent()` | `max_iterations=5` or higher |
| 4. Improve description | `description` field | Be specific about what the tool returns |

**Quick Fix** - Change the agent initialization to allow more iterations:


In [47]:
# =============================================================================
# 🔧 FIX: Improved Custom Tool with Better Retrieval
# =============================================================================
# This improved version addresses the iteration limit issue

class ImprovedSearchTool(BaseTool):
    """Improved restaurant search tool with better retrieval and formatting."""
    
    name: str = "restaurant search"
    
    # FIX 1: More descriptive - tells agent what to expect
    description: str = """Search Bella Vista restaurant knowledge base.
    Use for: hours, menu, reservations, ambiance, policies.
    Returns FAQ answers - extract the information needed to answer the user."""

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # FIX 2: Retrieve more documents (6 instead of default 4)
        store = vectorstore.as_retriever(search_kwargs={"k": 6})
        docs = store.invoke(query)
        
        if not docs:
            return "No information found."
        
        # FIX 3: Format results clearly so agent can parse them
        results = [f"• {doc.page_content}" for doc in docs]
        return "Restaurant Information:\n" + "\n".join(results)
    
    async def _arun(self, query: str, run_manager=None) -> str:
        raise NotImplementedError("Async not supported")

# FIX 4: Create agent with MORE iterations
improved_tools = [ImprovedSearchTool()]
improved_agent = initialize_agent(
    improved_tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iterations=5,  # Increased from 3 to 5
    handle_parsing_errors=True
)


In [48]:
# =============================================================================
# Test the Improved Agent
# =============================================================================
# Run the same query with the improved agent - should get better results!

improved_agent.invoke(input="When does the restaurant open?")




> Entering new AgentExecutor chain...
I need to find out the opening hours for the restaurant.  
Action: restaurant search  
Action Input: "What are the opening hours of the restaurant?"  
Observation: Restaurant Information:
• Q: What are the hours of operation for Bella Vista?
• Sundays, we welcome guests from 12 p.m. to 10 p.m.
• during weekends and holidays, to ensure a seamless dining experience.
• A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends
• Q: Do you offer vegetarian or vegan options at Bella Vista?
• Q: What type of cuisine does Bella Vista serve?
Thought:I have found the opening hours for the restaurant.  
Final Answer: Bella Vista opens on Sundays from 12 p.m. to 10 p.m.

> Finished chain.


{'input': 'When does the restaurant open?',
 'output': 'Bella Vista opens on Sundays from 12 p.m. to 10 p.m.'}

### Original Agent (for comparison - may hit iteration limit)


In [46]:
# =============================================================================
# Test 2: In-Scope Question (Uses Restaurant Tool)
# =============================================================================
# This question should trigger the restaurant search tool
# Watch the Thought -> Action -> Observation cycle in the output
#
# The agent will:
# 1. Recognize this is about the restaurant
# 2. Call the "restaurant search" tool
# 3. Parse the results
# 4. Formulate an answer (or keep searching)

agent.invoke(input="When does the restaurant open?")



> Entering new AgentExecutor chain...
I need to find out the opening hours of the restaurant.  
Action: restaurant search  
Action Input: "restaurant opening hours"  
Observation: during weekends and holidays, to ensure a seamless dining experience.
Sundays, we welcome guests from 12 p.m. to 10 p.m.
A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends
Q: What are the hours of operation for Bella Vista?
Thought:I need to find out the hours of operation for Bella Vista.  
Action: restaurant search  
Action Input: "Bella Vista hours of operation"  
Observation: Q: What are the hours of operation for Bella Vista?
dinner or a casual meal with friends, Bella Vista provides the perfect atmosphere.
Q: Do I need a reservation for Bella Vista?
Q: Is Bella Vista family-friendly?
Thought:I need to find out if Bella Vista is family-friendly.  
Action: restaurant search  
Action Input: "Is Bella Vista family-friendly?"  
Observation: Q: Is Bella Vista

{'input': 'When does the restaurant open?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

---

## Section 3: Conversational Agents with Memory

### Why Add Memory?

The agents we've built so far are **stateless** - they don't remember previous interactions. For chatbot-like applications, we need **conversational agents** that:
- Remember the conversation history
- Can refer back to previous topics
- Maintain context across multiple turns

### Key Differences from Stateless Agents

| Feature | Stateless Agent | Conversational Agent |
|---------|-----------------|---------------------|
| Memory | None | Conversation history |
| Context | Single query | Full conversation |
| Agent Type | `ZERO_SHOT_REACT_DESCRIPTION` | `CHAT_CONVERSATIONAL_REACT_DESCRIPTION` |
| Use Case | One-off questions | Multi-turn chat |

In [38]:
# =============================================================================
# Creating a Conversational Agent with Memory
# =============================================================================
# This agent maintains conversation history and can reference previous messages

from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

# Create a memory object to store conversation history
# memory_key="chat_history" - The key used in the prompt template
# return_messages=True - Store as message objects (required for chat models)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Create the conversational agent
# Key differences from before:
# 1. agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION (chat-optimized)
# 2. memory=memory (adds conversation history)
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  # Chat-optimized agent
    verbose=True, 
    memory=memory,                # Attach conversation memory
    handle_parsing_errors=True
)

In [39]:
# =============================================================================
# Inspecting the Chat Agent's Prompt Template
# =============================================================================
# The conversational agent uses a different prompt structure with multiple 
# message templates instead of a single string template

# Print the prompt messages (list of message templates)
print(agent_chain.agent.llm_chain.prompt.messages)

# =============================================================================
# The output shows:
# 1. SystemMessagePromptTemplate - Instructions for the AI assistant
# 2. MessagesPlaceholder("chat_history") - Where conversation history goes
# 3. HumanMessagePromptTemplate - Where the current user input goes
# 4. MessagesPlaceholder("agent_scratchpad") - Agent's reasoning steps
# =============================================================================

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations a

### Demonstrating Conversational Memory

Let's have a multi-turn conversation to see how the agent maintains context. Pay attention to:
1. How the agent remembers the previous topic
2. How "there" refers back to "Paris" from the previous question
3. How the chat_history grows with each interaction


In [41]:
# =============================================================================
# Turn 1: Initial Question
# =============================================================================
# First message in the conversation - establishes context about France/Paris
# The agent answers directly without needing to use any tools

result = agent_chain.invoke(input="What is the capital of France?")

# Notice the output contains:
# - 'input': The current question
# - 'chat_history': List of previous messages (will grow)
# - 'output': The agent's response
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The capital of France is Paris."
}
```

> Finished chain.


{'input': 'What is the capital of France?',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={})],
 'output': 'The capital of France is Paris.'}

In [ ]:
# =============================================================================
# Turn 2: Follow-up Question (Tests Memory!)
# =============================================================================
# Here's where memory matters: "there" refers to Paris!
# Without memory, the agent wouldn't know what "there" means
# 
# The agent correctly understands we're still talking about Paris

result = agent_chain.invoke(input="Any suggestions what to visit there?")

# Check the chat_history - it now contains 2 exchanges (4 messages total)
print(f"Conversation now has {len(result['chat_history'])} messages")
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "In Paris, you might want to visit iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and Montmartre. You can also explore beautiful parks like the Luxembourg Gardens and enjoy a stroll along the Seine River. Don't forget to try the local cuisine in charming cafés and restaurants!"
}
```

> Finished chain.


{'input': 'Any suggestions what to visit there?',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Any suggestions what to visit there?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="In Paris, you might want to visit iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and Montmartre. You can also explore beautiful parks like the Luxembourg Gardens and enjoy a stroll along the Seine River. Don't forget to try the local cuisine in charming cafés and restaurants!", additional_kwargs={}, response_metadata={})],
 'output': "In Paris, you might want to visit iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and Montmartre. You can also explore beautiful parks like the Luxembourg Gardens and enjoy a stroll along the S

In [42]:
# =============================================================================
# Turn 3: Topic Switch
# =============================================================================
# The agent can handle topic changes while maintaining full conversation history
# It understands we're now asking about places to visit in China
# (based on the context of previous travel-related questions)

result = agent_chain.invoke(input="What about china")

# The full conversation history is preserved
# This is useful for:
# - Understanding user intent from context
# - Providing personalized responses
# - Maintaining coherent multi-turn dialogues
print(f"Final conversation has {len(result['chat_history'])} messages")
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "China is a country in East Asia, known for its rich history, diverse culture, and significant global influence. The capital of China is Beijing."
}
```

> Finished chain.
Final conversation has 6 messages


{'input': 'What about china',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What about china', additional_kwargs={}, response_metadata={}),
  AIMessage(content='China is a country in East Asia, known for its rich history, diverse culture, and significant global influence. The capital of China is Beijing.', additional_kwargs={}, response_metadata={})],
 'output': 'China is a country in East Asia, known for its rich history, diverse culture, and significant global influence. The capital of China is Beijing.'}

In [ ]:
# =============================================================================
# 🎯 EXERCISE: Try It Yourself!
# =============================================================================
# 
# Exercise 1: Create a simple tool using the @tool decorator
# 
# from langchain.tools import tool
# 
# @tool
# def get_weather(city: str) -> str:
#     """Get the current weather for a city."""
#     # In a real app, you'd call a weather API
#     return f"The weather in {city} is sunny, 72°F"
# 
# Exercise 2: Add the weather tool to our agent and test it
# 
# Exercise 3: Create an agent with multiple tools (calculator + restaurant + weather)
#
# =============================================================================


In [ ]:
# Your code here - try creating your own tools and agents!


## Summary & Key Takeaways

### What We Learned

1. **Agents** use LLMs as reasoning engines to decide which tools to use
2. **Tools** extend agent capabilities with domain-specific functions
3. **ReAct Pattern** (Thought → Action → Observation) is the foundation
4. **Custom Tools** can be created by subclassing `BaseTool`
5. **Conversational Agents** maintain memory across multiple turns

### Agent Types Comparison

| Agent Type | Memory | Best For |
|------------|--------|----------|
| `ZERO_SHOT_REACT_DESCRIPTION` | ❌ | One-off questions |
| `CHAT_CONVERSATIONAL_REACT_DESCRIPTION` | ✅ | Multi-turn chat |
| `OPENAI_FUNCTIONS` | Optional | Reliable tool calling |
| `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` | ❌ | Complex tool inputs |

### Best Practices

1. **Tool Descriptions Matter**: Write clear, specific descriptions
2. **Handle Errors**: Use `handle_parsing_errors=True`
3. **Set Limits**: Use `max_iterations` to prevent infinite loops
4. **Choose Right Agent**: Match agent type to your use case

### What's Next?

- Explore **LangGraph** for more advanced agent workflows
- Learn about **Agent Tool Calling** with OpenAI Functions
- Build **Multi-Agent Systems** for complex tasks

---

*For the latest best practices, check the [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)*
